### Intro

Breadth first search (BFS), also known as a level order traversal, is a graph traversal algorithm most often used in interviews to find the shortest path between two vertices in an unweighted graph. BFS questions are my favorite to recieve in interviews because the interview prompt almost always gives it away that you want to use a BFS by mentioning a word like "shortest" or "least" and because BFS is very straightforward to implement. Typically the only tricky part of a BFS is determining how to generate the current node's neighbors, although this is usually very obvious. In a 2-dimensional array for instance a grid cell's neighbors are most often the adjacent grid cells.

### Template

In [ ]:
from collections import deque                    # use deque to implement a queue in python with O(1) FIFO operations

queue = deque([start_node])                      # initiate the queue
visited = set([start_node])                      # track visited nodes using a set to avoid revisiting nodes in a cyclic graph
depth = 0                                        # track the depth the BFS reaches

while queue:                                     # while the queue exists, traverse another level
    for _ in range(len(queue)):                  # crucial line, pop the queue as many times as there are nodes in the current level
        curr_node = queue.popleft()
        if curr_node == target_node:             # stop condition, if we've found what we're looking for, return the depth
            return depth

        for neighbor in curr_node.neighbors:     # otherwise iterate through curr_node's neighbor nodes
            if neighbor not in visited:
                queue.append(neighbor)           # append neighbor to next level
                visited.add(neighbor)            # add neighbor to visited set now so that it isn't readded to queue by future node

    depth += 1                                   # after traversing current level, increment depth

return -1                                        # if we never hit our stop condition, return unsuccessful search

### Examples

**[LC 752. Open the Lock](https://leetcode.com/problems/open-the-lock/) [Medium]**

This is a classic LeetCode problem where you're given a 4 digit lock and are tasked with finding the least amount of lock turns needed to get from combo `'0000'` to `target` without hitting a combo in `deadends` along the way. A lock turn consists of turning one wheel by one slot e.g. `'0000'` -> `'0001'`. Digits consist of the numbers 0-9.

You can think of this as an undirected graph problem where the vertices are valid lock combos and the edges connect lock combos that can be reached in one turn from one another. The fact that we are searching for the least number of lock turns tells us BFS is a great candidate for solving this problem since the first time we reach the `target` combo during our BFS traversal must be in the least number of turns.

Note: you'll notice the solution implements the `neighbors()` function as a generator which is always good practice to do in an interview. 

In [ ]:
from collections import deque

def openLock(deadends: list[str], target: str) -> int:
    
    def neighbors(combo: str):                          # generator for yeilding neighboring lock combos given a lock combo
        for i in range(4):                              # for each lock digit
            x = int(combo[i])
            for d in (-1, 1):                           # for each direction the lock can be spun by one unit
                y = (x + d) % 10                        # mod by 10 since there are 10 digits, for when we spin to -1 or to 10
                yield combo[:i] + str(y) + combo[i+1:]

    queue = deque(['0000'])
    visited = set(deadends + ['0000'])                  # combine deadends with visited combos since we don't want to visit/revisit either
    depth = 0

    while queue:
        for _ in range(len(queue)):
            curr_combo = queue.popleft()
            if curr_combo == target:
                return depth

            for nei_combo in neighbors(curr_combo):
                if nei_combo not in visited:
                    queue.append(nei_combo)
                    visited.add(nei_combo)

        depth += 1

    return -1                                           # return -1 if impossible to reach target combo

**[LC 286. Walls and Gates](https://leetcode.com/problems/walls-and-gates/) [Medium]**

Walls and gates is another classic LeetCode problem where you're given a 2-dimensional array where values in the array represent walls (`-1`), gates (`0`), or empty rooms (`2**31 - 1`). Our task is to modify the input array in place by filling in each empty room with an integer representing the length of the path to the room's nearest gate.

Again you see the word "nearest" so you consider if this problem can be modeled as a graph. 2-dimensional array problems are quite common on LeetCode and can be thought of as graphs in the sense that each cell in the grid is a vertex, while edge's connect the cells adjacent neighbors. In this problem, paths to gates must be 4-directional (meaning a neighboring cell can be north, south, east, or west of the current cell, this is different from 8-directional where neighbors can additionally be north-east, north-west, south-east, south-west of the current cell).

With this in mind your first thought might be to traverse the array and run a BFS every time we encounter a gate. While this is a valid strategy it would result in running multiple BFS's. A more optimal way of solving this problem is instead traversing the entire array and recording the coordinates of every gate. We then run a single BFS using the prerecorded gate coordinates as the starting nodes in the 1st level of our BFS. This way we only need to run a BFS once!

Note: there's no need for tracking a `visited` set in this problem since we already know visited cells have a value not equal to `2**31 - 1`.

In [ ]:
from collections import deque

def wallsAndGates(self, rooms: list[list[int]]) -> None:
    m, n = len(rooms), len(rooms[0])                               # record dims of the input array
    queue = deque()
    for i in range(m):                                             # iterate through the array and record coords of every gate
        for j in range(n):
            if rooms[i][j] == 0:
                queue.append((i,j))
                
    depth = 1
    
    while queue:                                                   # run a single BFS starting with the gates found earlier
        for _ in range(len(queue)):
            curr_room = queue.popleft()
            i, j = curr_room
            for nei_room in [(i-1,j), (i,j+1), (i+1,j), (i,j-1)]:  # iterate through 4-directional neighbor rooms
                nei_i, nei_j = nei_room
                if 0 <= nei_i < m and 0 <= nei_j < n and rooms[nei_i][nei_j] == 2**31 - 1:  # filter for valid neighbors
                    rooms[nei_i][nei_j] = depth
                    queue.append(nei_room)

        depth += 1

**[LC 127. Word Ladder](https://leetcode.com/problems/word-ladder/) [Hard]**

This one I actually got in a FAANG interview, it's labeled hard on LeetCode but shouldn't be too bad now that you understand the BFS pattern. The problem asks us what's the shortest way to get from a starting word `beginWord` to a target word `endWord`. At each step we can only change one letter in our current word, and the new resulting word must be in our word dictionary `wordList`. Return `0` if it's not possible to reach `endWord`.

Treat the words as vertices, each word's neighbors are all the possible words you can get by changing any letter in the current word to any other letter in the alphabet with the added caveat that the new word must be a word in `wordList`.

In [ ]:
from collections import deque

class Solution:
    def ladderLength(self, beginWord: str, endWord: str, wordList: list[str]) -> int:
        
        def neighbors(word: str):                          # generator for yeilding neighboring words given a word
            letters = 'abcdefghijklmnopqrstuvwxyz'
            for i in range(len(word)):                     # for each letter in given word
                for l in letters:                          # for each letter in the alphabet
                    yield word[:i] + l + word[i+1:]        # construct candidate neighbor word
        
        queue = deque([beginWord])
        visited = set([beginWord])
        wordList = set(wordList)                           # convert wordList to a set so we can search it in O(1)
        depth = 1
        
        while queue:
            for _ in range(len(queue)):
                curr_word = queue.popleft()
                if curr_word == endWord:
                    return depth

                for nei_word in neighbors(curr_word):
                    if nei_word in wordList and nei_word not in visited:  # check if neighbor word is a valid neighbor to visit
                        queue.append(nei_word)
                        visited.add(nei_word)

            depth += 1

        return 0